# 线段树
给定一个数组，需要对指定区间进行三种任务(操作):对区间里的数均加m(不返回数组)，把区间里的数均变成m(不返回数组)，计算区间里的数的所有和(返回结果)。难点是：要求以上操作时间复杂度为O($logN$)。注意到线段树中其实我们不需要操作原数组，这些前两个操作只是为了让我们返回出最后的累加和

分析：
- 求和需求：先把指定的arr(长度为N)，二分成所有区间的信息(求和)，用一个长度为4N的一维数组sum来表示，将其变成一个满二叉树。这个sum数组(0位置弃而不用)的特点是第i个位置的左节点位置是2i，右节点为2i+1。
- 加数需求：通过lazy机制，实现任务(在指定区间上加m)下发到子树上，这个下发机制使得任务下发完成，只有logN的时间复杂度。这个机制有个操作需要注意的是，如果来了一个自身“揽”不住的任务时但自己本身已经揽有任务了，需要先将已经揽得任务下发。
- 更新需求：也是通过lazy机制，但是更新任务来的被“揽”住的时候，该节点已经“揽”的所有加数需求全部抛弃。
- 下发函数：在加数需求和更新需求中我们都看到需要将已经“揽”的任务下发到子树，这里要主要先下发更新需求再下发加数需求。

In [ ]:
origin = [3, 4, 3, 4]#2, 6, 9, 8, 4, 9
MAXlEN = len(origin)+1
Sum = [0 for i in range(4 * MAXlEN)]
lazy = [0 for i in range(4 * MAXlEN)]
change= [None for i in range(4 * MAXlEN)]

# 汇总函数,汇总当前值
def pushUp(rt):
    Sum[rt] = Sum[2*rt] + Sum[2*rt+1]

# 下发本级已经存在的信息，下发顺序是先下发更新任务，再下发求和任务：
# l,r表示当前范围，rt表示当前范围的位置
def pushDown(l, r, rt):
    mid = (l+r)//2
    # 如果当前rt有更新任务，则下发
    if change[rt] is not None:
        updata(l, mid, change[rt], l, mid, rt*2, )
        updata(mid+1, r, change[rt], mid+1, r, rt*2+1)
        lazy[rt] = 0
        change[rt] = None

    # 如果当前rt有求和任务，则下发
    if lazy[rt] != 0:
        add(l, mid, lazy[rt], l, mid, rt*2)
        add(mid+1, r, lazy[rt], mid+1, r, rt*2+1)
        lazy[rt] = 0
        # 写法2
        # lazy[rt*2] += lazy[rt]
        # lazy[rt*2+1] += lazy[rt]
        # Sum[rt*2] += lazy[rt] * (mid-l+1)
        # Sum[rt*2+1] += lazy[rt] * (r-mid)
        # lazy[rt] = 0

# 通过递归构建sum数组,使得初始化Sum列表,l表示当前任务左区间，r表示当前任务有区间，rt表示在线段树的哪里。
def build(l, r, rt):
    if l == r:
        Sum[rt] = arr[l]
        return
    mid = (l+r)//2
    build(l, mid, 2*rt) # 建立左子树
    build(mid+1, r, 2*rt+1)   # 建立右子树
    pushUp(rt)  #汇总函数

# add任务，对指定区间上每个数都加一个数
# L,R表示任务区间;l,r表示当前rt的区间;value表示需要每个数加的值
def add(L, R, value, l, r, rt):
    if L<=l and r<=R:
        lazy[rt] += value
        Sum[rt] += value*(r-l+1)    #更新现在的sum信息
        return
    # 如果没有“揽住”([L,R]>[l,r]称为"揽住")，则需要把信息下发
    # 下发之前需要先把本级已经“揽住”的信息先下发
    pushDown(l, r, rt)
    mid = (l+r)//2
    if L <= mid:  #左节点是否需要接受信息
        add(L, R, value, l, mid, 2 * rt)    #下发到左节点
    if mid+1 <= R:  #右节点是否需要接受信息
        add(L, R, value, mid+1, r, 2 * rt + 1)    #下发到右节点
    pushUp(rt)  # 信息下发完成之后更新我的Sum


# 更新任务，对指定区间上所有数字都变成value
# L,R表示任务区间;l,r表示当前rt的区间;value表示需要需要变成的数
def updata(L, R, value, l, r, rt):
    if L<=l and r<=R:
        change[rt] = value 
        Sum[rt] = value * (r-l+1)  #之前的累加和全部无效了
        lazy[rt] = 0    #由于有更新任务因此之前积累的所有lazy信息都无效了
        return
    pushDown(l, r, rt)
    mid = (l+r)//2
    if L <= mid:  #左节点是否需要接受信息
        updata(L, R, value, l, mid, 2 * rt)    #下发到左节点
        
    if mid+1 <= R:  #右节点是否需要接受信息
        updata(L, R, value, mid+1, r, 2 * rt + 1)    #下发到右节点
    pushUp(rt)  # 信息下发完成之后更新我的Sum

# 查询函数，对数据进行查询操作。
def query(L, R, l, r, rt):
    if L<=l and r<=R:
        return Sum[rt]
    pushDown(l, r, rt)
    mid = (l+r)//2
    ans = 0
    if L <= mid:  
        ans += query(L, R, l, mid, 2 * rt)    #向左节点要信息
    if mid+1 <= R:  
        ans += query(L, R, mid+1, r, 2 * rt + 1)    #向右节点要信息
    return ans


In [ ]:
origin = [3, 4, 3, 4]#2, 6, 9, 8, 4, 9
MAXlEN = len(origin)+1
Sum = [0 for i in range(4 * MAXlEN)]
lazy = [0 for i in range(4 * MAXlEN)]
change= [None for i in range(4 * MAXlEN)]


# 重建一个数组,将原始数组放进arr中,0位置舍弃
arr = [None for i in range(MAXlEN)] 
for i in range(1, MAXlEN):
    arr[i] = origin[i-1]
build(1, len(origin), 1)
add(1, 3, 5, 1,len(origin),1)  #第1到3的数都加5
updata(1, 3, 0, 1,len(origin),1)  #第1到3的数全变成0
updata(2, 3, 1, 1,len(origin),1)  #第2到3的数全变成1
add(1, 4, 2, 1, len(origin), 1)  #第1到4的数都加2
query(1, 3, 1, len(origin), 1) #查询1-3的数

## 封装类
- 错误点1：类函数要时刻注意self的拼写，如果self拼写错误，该函数将看不到self.变量报错
- 错误点2：类函数的定义中，如果要定义默认值，默认值不能为self.变量，原因是在函数定义时看不到self.变量，解决方法是将默认值写为None,并在函数里面采用条件判读语句self.变量 if r is None else r来赋值为self.变量

In [ ]:
class SegmentTree():
    def __init__(self, origin):
        self.MAXlEN = len(origin)+1
        self.arr = [None for i in range(self.MAXlEN)] 
        self.Sum = [0 for i in range(4 * self.MAXlEN)]
        self.lazy = [0 for i in range(4 * self.MAXlEN)]
        self.change= [None for i in range(4 * self.MAXlEN)]
        for i in range(1, self.MAXlEN):
            self.arr[i] = origin[i-1]
        self.build(1, len(origin), 1)
        

    # 汇总函数,汇总当前值
    def pushUp(self, rt):
        self.Sum[rt] = self.Sum[2*rt] + self.Sum[2*rt+1]

    # 下发本级已经存在的信息，下发顺序是先下发更新任务，再下发求和任务：
    # l,r表示当前范围，rt表示当前范围的位置
    def pushDown(self, l, r, rt):
        mid = (l+r)//2
        # 如果当前rt有更新任务，则下发
        if self.change[rt] != None:
            self.updata(l, mid, self.change[rt], l, mid, rt*2, )
            self.updata(mid+1, r, self.change[rt], mid+1, r, rt*2+1)
            self.lazy[rt] = 0
            self.change[rt] = None

        # 如果当前rt有求和任务，则下发
        if self.lazy[rt] != 0:
            self.add(l, mid, self.lazy[rt], l, mid, rt*2)
            self.add(mid+1, r, self.lazy[rt], mid+1, r, rt*2+1)
            self.lazy[rt] = 0
            # 写法2
            # lazy[rt*2] += lazy[rt]
            # lazy[rt*2+1] += lazy[rt]
            # Sum[rt*2] += lazy[rt] * (mid-l+1)
            # Sum[rt*2+1] += lazy[rt] * (r-mid)
            # lazy[rt] = 0

    # 通过递归构建sum数组,使得初始化Sum列表,l表示当前任务左区间，r表示当前任务有区间，rt表示在线段树的哪里。
    def build(self, l, r, rt):
        if l == r:
            self.Sum[rt] = self.arr[l]
            return
        mid = (l+r)//2
        self.build(l, mid, 2*rt)    # 建立左子树
        self.build(mid+1, r, 2*rt+1)   # 建立右子树
        self.pushUp(rt)  #汇总函数

    # add任务，对指定区间上每个数都加一个数
    # L,R表示任务区间;l,r表示当前rt的区间;value表示需要每个数加的值
    def add(self, L, R, value, l=1, r=None, rt=1):
        r = self.MAXlEN-1 if r is None else r
        if L<=l and r<=R:
            self.lazy[rt] += value
            self.Sum[rt] += value*(r-l+1)    #更新现在的sum信息
            return
        # 如果没有“揽住”([L,R]>[l,r]称为"揽住")，则需要把信息下发
        # 下发之前需要先把本级已经“揽住”的信息先下发
        self.pushDown(l, r, rt)
        mid = (l+r)//2
        if L <= mid:  #左节点是否需要接受信息
            self.add(L, R, value, l, mid, 2 * rt)    #下发到左节点
        if mid+1 <= R:  #右节点是否需要接受信息
            self.add(L, R, value, mid+1, r, 2 * rt + 1)    #下发到右节点
        self.pushUp(rt)  # 信息下发完成之后更新我的Sum


    # 更新任务，对指定区间上所有数字都变成value
    # L,R表示任务区间;l,r表示当前rt的区间;value表示需要需要变成的数
    def updata(self, L, R, value, l=1, r=None, rt=1):
        r = self.MAXlEN-1 if r is None else r
        if L<=l and r<=R:
            self.change[rt] = value 
            self.Sum[rt] = value * (r-l+1)  #之前的累加和全部无效了
            self.lazy[rt] = 0    #由于有更新任务因此之前积累的所有lazy信息都无效了
            return
        self.pushDown(l, r, rt)
        mid = (l+r)//2
        if L <= mid:  #左节点是否需要接受信息
            self.updata(L, R, value, l, mid, 2 * rt)    #下发到左节点
        if mid+1 <= R:  #右节点是否需要接受信息
            self.updata(L, R, value, mid+1, r, 2 * rt + 1)    #下发到右节点
        self.pushUp(rt)  # 信息下发完成之后更新我的Sum

    # 查询函数，对数据进行查询操作。
    def query(self, L, R, l=1, r=None, rt=1):
        r = self.MAXlEN-1 if r is None else r
        if L<=l and r<=R:
            return self.Sum[rt]
        self.pushDown(l, r, rt)
        mid = (l+r)//2
        ans = 0
        if L <= mid:  
            ans += self.query(L, R, l, mid, 2 * rt)    #向左节点要信息
        if mid+1 <= R:  
            ans += self.query(L, R, mid+1, r, 2 * rt + 1)    #向右节点要信息
        return ans


In [ ]:
origin = [3, 4, 3, 4]#2, 6, 9, 8, 4, 9
linetree = SegmentTree(origin)
linetree.add(1,3,5)
linetree.updata(1,3,0)
linetree.updata(2,3,1)
linetree.add(1,4,2)
linetree.query(1,3)